In [16]:
# from langchain_experimental.sql import SQLDatabaseChain
from langchain_community.utilities.sql_database import SQLDatabase
from sqlalchemy import create_engine, Table, Column, Integer, String, MetaData
from sqlalchemy.sql import text

In [25]:


engine = create_engine('sqlite:///participants.db')
metadata = MetaData()

# Define the participants table
participants_table = Table(
    'participants', metadata,
    Column('sp_id', Integer, primary_key=True),
    Column('age', Integer),
    Column('gender', String),
    Column('years_of_experience', Integer),
    Column('skills', String),
    Column('education_specialization', String),
    Column('past_jobs', String)
)

# Create the table
metadata.create_all(engine)
def populate_fake_data():
    fake_data = [
        {"sp_id": 1, "age": 35, "gender": "Male", "years_of_experience": 10, "skills": "Backend Development, Python, APIs", "education_specialization": "Computer Science", "past_jobs": "Software Engineer at XYZ"},
        {"sp_id": 2, "age": 28, "gender": "Female", "years_of_experience": 5, "skills": "Frontend Development, React, CSS", "education_specialization": "Information Technology", "past_jobs": "UI Developer at ABC"},
        {"sp_id": 3, "age": 40, "gender": "Male", "years_of_experience": 15, "skills": "DevOps, CI/CD, Kubernetes", "education_specialization": "Systems Engineering", "past_jobs": "DevOps Engineer at DEF"},
        {"sp_id": 4, "age": 32, "gender": "Female", "years_of_experience": 8, "skills": "Data Science, Machine Learning, R", "education_specialization": "Data Science", "past_jobs": "Data Scientist at GHI"},
        {"sp_id": 5, "age": 45, "gender": "Male", "years_of_experience": 20, "skills": "Database Management, SQL, Oracle", "education_specialization": "Database Administration", "past_jobs": "DBA at JKL"},
        {"sp_id": 6, "age": 29, "gender": "Female", "years_of_experience": 6, "skills": "Mobile Development, Swift, Android", "education_specialization": "Mobile Computing", "past_jobs": "Mobile Developer at MNO"},
        {"sp_id": 7, "age": 38, "gender": "Male", "years_of_experience": 12, "skills": "Cybersecurity, Ethical Hacking, Firewalls", "education_specialization": "Cybersecurity", "past_jobs": "Security Analyst at PQR"},
        {"sp_id": 8, "age": 27, "gender": "Female", "years_of_experience": 4, "skills": "Cloud Computing, AWS, Azure", "education_specialization": "Cloud Technology", "past_jobs": "Cloud Engineer at STU"},
        {"sp_id": 9, "age": 36, "gender": "Male", "years_of_experience": 14, "skills": "AI Research, NLP, TensorFlow", "education_specialization": "Artificial Intelligence", "past_jobs": "AI Specialist at VWX"},
        {"sp_id": 10, "age": 33, "gender": "Female", "years_of_experience": 9, "skills": "Project Management, Agile, Scrum", "education_specialization": "Business Administration", "past_jobs": "Project Manager at YZA"}
    ]

    with engine.begin() as conn:
        conn.execute(participants_table.delete())
        conn.execute(participants_table.insert(), fake_data)

populate_fake_data()

# sql_db = SQLDatabase(engine)

sql_query = """SELECT
        CASE
            WHEN age BETWEEN 0 AND 9 THEN '0-9'
            WHEN age BETWEEN 10 AND 19 THEN '10-19'
            WHEN age BETWEEN 20 AND 29 THEN '20-29'
            WHEN age BETWEEN 30 AND 39 THEN '30-39'
            WHEN age BETWEEN 40 AND 49 THEN '40-49'
            WHEN age BETWEEN 50 AND 59 THEN '50-59'
            WHEN age BETWEEN 60 AND 69 THEN '60-69'
            WHEN age BETWEEN 70 AND 79 THEN '70-79'
            WHEN age BETWEEN 80 AND 89 THEN '80-89'
            WHEN age BETWEEN 90 AND 99 THEN '90-99'
        END AS age_bucket,
        COUNT(*) AS count
    FROM participants
    GROUP BY age_bucket
    ORDER BY age_bucket ASC
"""
sql_query_input = text(sql_query)
with engine.begin() as conn:
    results = conn.execute(sql_query_input).fetchall()

In [26]:
results

[('20-29', 3), ('30-39', 5), ('40-49', 2)]

In [ ]:
def execute_sql_query_stuctured(query):
    if not query:
        return []

    prompt = f"""
    Generate a valid SQL query based on the following database schema and user request:

    Schema:
    {json.dumps({
        "participants": {
            "columns": [
                "sp_id", "age", "gender", "years_of_experience"
            ]
        }
    }, indent=2)}

    User Request: {query}

    SQL Query:
    """
    
    llm = OpenAI()
    response = llm(prompt)
    sql_query = response.strip()
    print(f"Generated SQL Query: \n{sql_query}")
    try:
        with engine.begin() as conn:
            results = conn.execute(text(sql_query)).fetchall()
        return [dict(r._mapping) for r in results]
    except Exception as e:
        print(f"Error executing SQL Query: {e}")
        return []

def chatbot_structured(query):
    sql_results = execute_sql_query_stuctured(query)
    llm = OpenAI()
    print(f"SQL Results: \n{sql_results}\n")
    prompt = f"""
    Based on the user's query: "{query}", answer the query by using the results generated by
    a SQL query on the participants data that is preparing the data for you to answer the users question.

    SQL Results to use to answer the user's query:
    {json.dumps(sql_results, default=str)}
    """
    response = llm(prompt)
    return response